# Tuned ANN Using Keras


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

%matplotlib inline
sns.set(color_codes=True)

In [2]:
Train_data=pd.read_csv("Train_UWu5bXk.csv")
Test_data=pd.read_csv("Test_u94Q5KV.csv")

In [3]:
Data=Train_data.append(Test_data,sort=False)

In [4]:
Data.isnull().sum()

Item_Identifier                 0
Item_Weight                  2439
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                  4016
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

In [5]:
Data['Item_Fat_Content'].value_counts()

Low Fat    8485
Regular    4824
LF          522
reg         195
low fat     178
Name: Item_Fat_Content, dtype: int64

In [6]:
Data['Item_Fat_Content'] = Data['Item_Fat_Content'].replace({'LF':'Low Fat','reg':'Regular','low fat':'Low Fat'})

In [7]:
Data.groupby('Item_Identifier')['Item_Weight'].mean().head(5)

Item_Identifier
DRA12    11.600
DRA24    19.350
DRA59     8.270
DRB01     7.390
DRB13     6.115
Name: Item_Weight, dtype: float64

In [12]:
Data[Data['Item_Identifier']=='FDW14']

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
4979,FDW14,8.3,Regular,0.038212,Dairy,87.7198,OUT046,1997,Small,Tier 1,Supermarket Type1,1569.9564
6270,FDW14,8.3,Regular,0.038367,Dairy,87.1198,OUT018,2009,Medium,Tier 3,Supermarket Type2,959.4178
8033,FDW14,8.3,Regular,0.038204,Dairy,87.7198,OUT035,2004,Small,Tier 2,Supermarket Type1,1831.6158
1,FDW14,8.3,Regular,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1,NaN
2982,FDW14,8.3,Regular,0.000000,Dairy,87.8198,OUT010,1998,NaN,Tier 3,Grocery Store,NaN
3911,FDW14,8.3,Regular,0.038271,Dairy,85.9198,OUT049,1999,Medium,Tier 1,Supermarket Type1,NaN
4243,FDW14,8.3,Regular,0.038180,Dairy,88.7198,OUT013,1987,High,Tier 3,Supermarket Type1,NaN
4478,FDW14,8.3,Regular,0.038289,Dairy,88.4198,OUT045,2002,NaN,Tier 2,Supermarket Type1,NaN
5142,FDW14,NaN,Regular,0.038026,Dairy,87.8198,OUT027,1985,Medium,Tier 3,Supermarket Type3,NaN


In [15]:
for i in Data.groupby('Item_Identifier')['Item_Weight'].mean().index:
    Data.loc[Data.loc[:,'Item_Identifier']==i,'Item_Weight']=Data.groupby('Item_Identifier')['Item_Weight'].mean()[i]

In [16]:
#Data['Outlet_Size'].fillna(Data['Outlet_Size'].mode()[0],inplace=True)
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Grocery Store'].mode()[0],inplace=True)

In [17]:
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type1'].mode()[0],inplace=True)

In [18]:
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type2'].mode()[0],inplace=True)

In [19]:
Data.Outlet_Size.fillna(Data.Outlet_Size[Data['Outlet_Type']=='Supermarket Type3'].mode()[0],inplace=True)

In [20]:
Data.isnull().sum()

Item_Identifier                 0
Item_Weight                     0
Item_Fat_Content                0
Item_Visibility                 0
Item_Type                       0
Item_MRP                        0
Outlet_Identifier               0
Outlet_Establishment_Year       0
Outlet_Size                     0
Outlet_Location_Type            0
Outlet_Type                     0
Item_Outlet_Sales            5681
dtype: int64

In [21]:
Train_data=Data.dropna()

In [22]:
Test_Data=Data[Data['Item_Outlet_Sales'].isnull()]
Test_Data.drop('Item_Outlet_Sales',axis=1,inplace=True)

/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


In [23]:
print(Test_Data.shape)
print(Train_data.shape)

(5681, 11)
(8523, 12)


In [24]:
from sklearn.preprocessing import LabelEncoder

In [25]:
categorical_list=['Item_Fat_Content','Item_Type','Outlet_Identifier','Outlet_Size','Outlet_Location_Type','Outlet_Type','Outlet_Establishment_Year']

In [26]:
le = LabelEncoder()
for i in categorical_list:
    Train_data[i]=le.fit_transform(Train_data[i])
    Train_data[i]=Train_data[i].astype('category')
    Test_Data[i]=le.fit_transform(Test_Data[i])
    Test_Data[i]=Test_Data[i].astype('category')

/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.
/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a 

# ANN

In [27]:
from keras.models import Sequential
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasRegressor


Using TensorFlow backend.


In [38]:
def c_model():
    # create model
    model = Sequential()
    model.add(Dense(78, input_dim=9, kernel_initializer='orthogonal', activation='relu'))
    model.add(Dense(65,activation='relu',kernel_initializer='orthogonal'))
    model.add(Dense(52,activation='relu',kernel_initializer='orthogonal'))
    model.add(Dense(39,activation='relu',kernel_initializer='orthogonal'))
    model.add(Dense(26,activation='relu',kernel_initializer='orthogonal'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam',metrics=['accuracy'])
    return model

In [39]:
model = KerasRegressor(build_fn=c_model, epochs=50, batch_size=32)

In [40]:
X_train=Train_data.drop(['Item_Outlet_Sales','Outlet_Identifier','Item_Identifier'],axis=1)
y_train=Train_data['Item_Outlet_Sales']

In [41]:
model.fit(X_train,y_train)

Epoch 1/50
8523/8523 [==============================] - 4s 415us/step - loss: 3222895.1142 - acc: 0.0000e+00
Epoch 2/50
8523/8523 [==============================] - 1s 112us/step - loss: 1953212.3980 - acc: 0.0000e+00
Epoch 3/50
8523/8523 [==============================] - 1s 110us/step - loss: 1921068.0771 - acc: 0.0000e+00
Epoch 4/50
8523/8523 [==============================] - 1s 118us/step - loss: 1870799.0590 - acc: 0.0000e+00
Epoch 5/50
8523/8523 [==============================] - 1s 114us/step - loss: 1815494.5580 - acc: 0.0000e+00
Epoch 6/50
8523/8523 [==============================] - 1s 112us/step - loss: 1726972.1035 - acc: 0.0000e+00
Epoch 7/50
8523/8523 [==============================] - 1s 113us/step - loss: 1635264.7386 - acc: 0.0000e+00
Epoch 8/50
8523/8523 [==============================] - 1s 128us/step - loss: 1561713.7422 - acc: 0.0000e+00
Epoch 9/50
8523/8523 [==============================] - 1s 141us/step - loss: 1493852.9727 - acc: 0.0000e+00
Epoch 10/50
8523/85

# Parameter Tuning using GridSearchCV

## Trying different batch_sizes and epochs:

In [42]:
from sklearn.model_selection import GridSearchCV

model = KerasRegressor(build_fn=c_model)

batch_sizes = [10, 20, 50, 100]
epochs = [5, 10, 50]
parameters = {'batch_size': batch_sizes, 'epochs': epochs}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

/home/ghost/anaconda3/envs/env2/lib/python3.7/site-packages/sklearn/model_selection/_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)


Epoch 1/5
5682/5682 [==============================] - 4s 757us/step - loss: 2541559.4193 - acc: 0.0000e+00
Epoch 2/5
5682/5682 [==============================] - 2s 323us/step - loss: 1938448.8186 - acc: 0.0000e+00
Epoch 3/5
5682/5682 [==============================] - 2s 313us/step - loss: 1869828.5038 - acc: 0.0000e+00
Epoch 4/5
5682/5682 [==============================] - 2s 312us/step - loss: 1727231.8520 - acc: 0.0000e+00
Epoch 5/5
5682/5682 [==============================] - 1s 226us/step
Epoch 1/5
5682/5682 [==============================] - 4s 778us/step - loss: 2492107.8896 - acc: 0.0000e+00
Epoch 2/5
5682/5682 [==============================] - 2s 329us/step - loss: 1974686.9477 - acc: 0.0000e+00
Epoch 3/5
5682/5682 [==============================] - 2s 327us/step - loss: 1884075.4074 - acc: 0.0000e+00
Epoch 4/5
5682/5682 [==============================] - 2s 328us/step - loss: 1756805.5988 - acc: 0.0000e+00
Epoch 5/5
5682/5682 [==============================] - 1s 233us/ste

5682/5682 [==============================] - 3s 520us/step - loss: 1177484.8148 - acc: 0.0000e+00
Epoch 48/50
5682/5682 [==============================] - 3s 512us/step - loss: 1176093.1648 - acc: 0.0000e+00
Epoch 49/50
5682/5682 [==============================] - 3s 520us/step - loss: 1187117.8601 - acc: 0.0000e+00
Epoch 50/50
5682/5682 [==============================] - 2s 404us/step
Epoch 1/50
5682/5682 [==============================] - 7s 1ms/step - loss: 2463816.6613 - acc: 0.0000e+00
Epoch 2/50
5682/5682 [==============================] - 3s 531us/step - loss: 1907670.4795 - acc: 0.0000e+00
Epoch 3/50
5682/5682 [==============================] - 3s 538us/step - loss: 1833056.5481 - acc: 0.0000e+00
Epoch 4/50
5682/5682 [==============================] - 3s 528us/step - loss: 1754087.9083 - acc: 0.0000e+00
Epoch 5/50
5682/5682 [==============================] - 3s 526us/step - loss: 1631830.7919 - acc: 0.0000e+00
Epoch 6/50
5682/5682 [==============================] - 3s 528us/ste

5682/5682 [==============================] - 1s 254us/step - loss: 1171074.9500 - acc: 0.0000e+00
Epoch 43/50
5682/5682 [==============================] - 2s 291us/step - loss: 1162570.6252 - acc: 0.0000e+00
Epoch 44/50
5682/5682 [==============================] - 2s 299us/step - loss: 1162479.1519 - acc: 0.0000e+00
Epoch 45/50
5682/5682 [==============================] - 2s 304us/step - loss: 1176982.2184 - acc: 0.0000e+00
Epoch 46/50
5682/5682 [==============================] - 2s 295us/step - loss: 1157871.4641 - acc: 0.0000e+00
Epoch 47/50
5682/5682 [==============================] - 2s 299us/step - loss: 1174895.2234 - acc: 0.0000e+00
Epoch 48/50
5682/5682 [==============================] - 2s 299us/step - loss: 1165766.1771 - acc: 0.0000e+00
Epoch 49/50
5682/5682 [==============================] - 2s 301us/step - loss: 1163606.3833 - acc: 0.0000e+00
Epoch 50/50
5682/5682 [==============================] - 1s 243us/step
Epoch 1/50
5682/5682 [==============================] - 6s 1m

5682/5682 [==============================] - 1s 138us/step - loss: 1878695.4865 - acc: 0.0000e+00
Epoch 9/10
5682/5682 [==============================] - 1s 134us/step - loss: 1853420.2343 - acc: 0.0000e+00
Epoch 10/10
5682/5682 [==============================] - 1s 109us/step
Epoch 1/10
5682/5682 [==============================] - 6s 1ms/step - loss: 4702404.9084 - acc: 0.0000e+00
Epoch 2/10
5682/5682 [==============================] - 1s 142us/step - loss: 1937206.9560 - acc: 0.0000e+00
Epoch 3/10
5682/5682 [==============================] - 1s 135us/step - loss: 1918196.7752 - acc: 0.0000e+00
Epoch 4/10
5682/5682 [==============================] - 1s 143us/step - loss: 1892189.4336 - acc: 0.0000e+00
Epoch 5/10
5682/5682 [==============================] - 1s 141us/step - loss: 1870955.7526 - acc: 0.0000e+00
Epoch 6/10
5682/5682 [==============================] - 1s 143us/step - loss: 1835834.0665 - acc: 0.0000e+00
Epoch 7/10
5682/5682 [==============================] - 1s 140us/step 

5682/5682 [==============================] - 1s 146us/step - loss: 1231672.1555 - acc: 0.0000e+00
Epoch 33/50
5682/5682 [==============================] - 1s 143us/step - loss: 1224021.3195 - acc: 0.0000e+00
Epoch 34/50
5682/5682 [==============================] - 1s 145us/step - loss: 1218030.3230 - acc: 0.0000e+00
Epoch 35/50
5682/5682 [==============================] - 1s 146us/step - loss: 1219968.4531 - acc: 0.0000e+00
Epoch 36/50
5682/5682 [==============================] - 1s 145us/step - loss: 1226276.6757 - acc: 0.0000e+00
Epoch 37/50
5682/5682 [==============================] - 1s 143us/step - loss: 1205840.7570 - acc: 0.0000e+00
Epoch 38/50
5682/5682 [==============================] - 1s 148us/step - loss: 1217306.7187 - acc: 0.0000e+00
Epoch 39/50
5682/5682 [==============================] - 1s 148us/step - loss: 1231564.9425 - acc: 0.0000e+00
Epoch 40/50
5682/5682 [==============================] - 1s 142us/step - loss: 1200028.6547 - acc: 0.0000e+00
Epoch 41/50
5682/5682 

5682/5682 [==============================] - 0s 87us/step - loss: 1272049.0632 - acc: 0.0000e+00
Epoch 30/50
5682/5682 [==============================] - 0s 85us/step - loss: 1263288.1949 - acc: 0.0000e+00
Epoch 31/50
5682/5682 [==============================] - 0s 85us/step - loss: 1266981.5376 - acc: 0.0000e+00
Epoch 32/50
5682/5682 [==============================] - 0s 82us/step - loss: 1249795.7125 - acc: 0.0000e+00
Epoch 33/50
5682/5682 [==============================] - 0s 87us/step - loss: 1225212.5919 - acc: 0.0000e+00
Epoch 34/50
5682/5682 [==============================] - 0s 86us/step - loss: 1228650.4289 - acc: 0.0000e+00
Epoch 35/50
5682/5682 [==============================] - 0s 85us/step - loss: 1222184.4624 - acc: 0.0000e+00
Epoch 36/50
5682/5682 [==============================] - 0s 85us/step - loss: 1210579.0085 - acc: 0.0000e+00
Epoch 37/50
5682/5682 [==============================] - 0s 84us/step - loss: 1205466.4187 - acc: 0.0000e+00
Epoch 38/50
5682/5682 [========

GridSearchCV(cv='warn', error_score='raise-deprecating',
       estimator=<keras.wrappers.scikit_learn.KerasRegressor object at 0x7f1dc5c1d6a0>,
       fit_params=None, iid='warn', n_jobs=None,
       param_grid={'batch_size': [10, 20, 50, 100], 'epochs': [5, 10, 50]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [43]:
print(clf.best_score_, clf.best_params_)
means = clf.cv_results_['mean_test_score']
parameters = clf.cv_results_['params']
for mean, parammeter in zip(means, parameters):
    print(mean, parammeter)

-1175213.1255059838 {'batch_size': 20, 'epochs': 50}
-1643469.406366413 {'batch_size': 10, 'epochs': 5}
-1229566.6678628784 {'batch_size': 10, 'epochs': 10}
-1204661.2024858745 {'batch_size': 10, 'epochs': 50}
-1732788.9921031694 {'batch_size': 20, 'epochs': 5}
-1387016.53508004 {'batch_size': 20, 'epochs': 10}
-1175213.1255059838 {'batch_size': 20, 'epochs': 50}
-1929413.3798398452 {'batch_size': 50, 'epochs': 5}
-1704805.4671330517 {'batch_size': 50, 'epochs': 10}
-1206704.8211823888 {'batch_size': 50, 'epochs': 50}
-1951166.416989323 {'batch_size': 100, 'epochs': 5}
-1889530.3546726506 {'batch_size': 100, 'epochs': 10}
-1182522.7693593805 {'batch_size': 100, 'epochs': 50}


In [44]:
def c_model():
    # create model
    model = Sequential()
    model.add(Dense(78, input_dim=9, kernel_initializer='orthogonal', activation='relu'))
    model.add(Dense(65,activation='relu',kernel_initializer='orthogonal'))
    model.add(Dense(52,activation='relu',kernel_initializer='orthogonal'))
    model.add(Dense(39,activation='relu',kernel_initializer='orthogonal'))
    model.add(Dense(26,activation='relu',kernel_initializer='orthogonal'))
    model.add(Dense(1, kernel_initializer='normal'))
    # Compile model
    model.compile(loss='mean_squared_error', optimizer='adam')
    return model

model = KerasRegressor(build_fn=c_model, epochs=50, batch_size=32)
optimizers=['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']
parameters = {'optimizer':optimizers}
clf = GridSearchCV(model, parameters)
clf.fit(X_train, y_train)

ValueError: optimizer is not a legal parameter

In [ ]:
batch_sizes = [10, 20, 50, 100]
epochs = [5, 10, 50]
parameters = {'batch_size': batch_sizes, 'epochs': epochs}